# Pipeline of Digits

This is a starting notebook for solving the "Pipeline of Digits" assignment.


This notebook was created by [Santiago L. Valdarrama](https://twitter.com/svpino) as part of the [Machine Learning School](https://www.ml.school) program.

Let's make sure we are running the latest version of the SakeMaker's SDK. **Restart the notebook** after you upgrade the library.

In [2]:
# !pip install -q --upgrade awscli
# !pip install -q --upgrade pip
# !pip install -q --upgrade sagemaker
# !pip show sagemaker

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sagemaker 2.145.0 requires importlib-metadata<5.0,>=1.4.0, but you have importlib-metadata 6.3.0 which is incompatible.
aiobotocore 2.4.2 requires botocore<1.27.60,>=1.27.59, but you have botocore 1.29.118 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 4.0.1 requires pyqt5<5.13; python_version >= "3", which is not installed.
spyder 4.0.1 requires pyqtwebengine<5.13; python_version >= "3", which is not installed.
pytest-astropy 0.8.0 requires pytest-cov>=2.0, which is not installed.
pytest-astropy 0.8.0 requires pytest-filter-subpackage>=0.1, which is

In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
from botocore.exceptions import ClientError
from sagemaker.inputs import FileSystemInput
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.processing import ScriptProcessor
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.workflow.steps import ProcessingStep, TuningStep, TrainingStep
from sagemaker.workflow.model_step import ModelStep
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.workflow.parameters import ParameterInteger, ParameterString, ParameterFloat
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.steps import CacheConfig
from sagemaker.workflow.properties import PropertyFile


from sagemaker.tuner import HyperparameterTuner
from sagemaker.inputs import TrainingInput
from sagemaker.parameter import IntegerParameter
from sagemaker.parameter import ContinuousParameter
from sagemaker.inputs import TrainingInput
from sagemaker.tensorflow import TensorFlow ,TensorFlowProcessor, TensorFlowModel

from sagemaker import ModelPackage
from sagemaker.model import Model
from sagemaker.model_metrics import MetricsSource, ModelMetrics 
from sagemaker.predictor import Predictor
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo
from sagemaker.workflow.conditions import ConditionLessThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.fail_step  import FailStep
from sagemaker.workflow.functions import JsonGet
from sagemaker.workflow.functions import Join

from sagemaker.tensorflow.model import TensorFlowModel
from sagemaker.tensorflow.model import TensorFlowPredictor
from sagemaker.workflow.lambda_step import LambdaStep, LambdaOutput, LambdaOutputTypeEnum
from sagemaker.workflow.parameters import ParameterBoolean
from sagemaker.lambda_helper import Lambda
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
from sagemaker.s3 import S3Downloader


# from sagemaker.tensorflow import TensorFlowProcessor
# from sagemaker.workflow.steps import TuningStep
# from sagemaker.workflow.steps import TrainingStep



In [3]:
import boto3
import sagemaker
import pandas as pd

from pathlib import Path

iam_client = boto3.client("iam")
sagemaker_client = boto3.client("sagemaker")
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()

## Creating the S3 Bucket

Let's create an S3 bucket where you will upload all the information generated by the pipeline. Make sure you set `BUCKET` to the name of the bucket you want to use. This name has to be unique.

If you want to create a bucket in a region other than `us-east-1`, use this command instead:

```
!aws s3api create-bucket --bucket $BUCKET --create-bucket-configuration LocationConstraint=$region
```

The `LocationConstraint` argument should specify the region where you want to create the bucket.

In [4]:
# uncomment if you want create new bucket

BUCKET = "mlschool-mnist"
region = "eu-north-1"

In [5]:
# !aws s3api create-bucket --bucket $BUCKET --create-bucket-configuration LocationConstraint=$region

In [6]:
# import os
# current_folder = os.getcwd()
# print(current_folder)
# %ls

/root/ml-school/mnist
code/     dataset.tar.gz  mnist.ipynb      train.py
dataset/  evaluation.py   preprocessor.py


## Loading the dataset

We have two CSV files containing the MNIST dataset. These files come from the [MNIST in CSV](https://www.kaggle.com/datasets/oddrationale/mnist-in-csv) Kaggle dataset.

The `mnist_train.csv` file contains 60,000 training examples and labels. The `mnist_test.csv` contains 10,000 test examples and labels. Each row consists of 785 values: the first value is the label (a number from 0 to 9) and the remaining 784 values are the pixel values (a number from 0 to 255).

Let's extract the `dataset.tar.gz` file.

In [7]:
MNIST_FOLDER = "/root/ml-school/mnist"
DATASET_FOLDER = Path(MNIST_FOLDER) / "dataset"

!tar -xvzf $MNIST_FOLDER/dataset.tar.gz -C $MNIST_FOLDER --no-same-owner

dataset/
dataset/mnist_test.csv
dataset/mnist_train.csv


Let's load the first 10 rows of the test set.

In [8]:
df = pd.read_csv(DATASET_FOLDER / "mnist_train.csv", nrows=5)
df

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Uploading dataset to S3

In [10]:
%%writefile {MNIST_FOLDER}/preprocessor.py

import os
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from pickle import dump
from pathlib import Path


# This is the location where the SageMaker Processing job
# will save the input dataset.
BASE_DIR = "/opt/ml/processing"
DATA_FILEPATH_TRAIN = Path(BASE_DIR) / "input" / "mnist_train" / "mnist_train.csv"
DATA_FILEPATH_TEST = Path(BASE_DIR) / "input" / "mnist_test" / "mnist_test.csv"


def save_splits(base_dir, train, validation, test):
    """
    One of the goals of this script is to output the three
    dataset splits. This function will save each of these
    splits to disk.
    """

    train_path = Path(base_dir) / "train"
    validation_path = Path(base_dir) / "validation"
    test_path = Path(base_dir) / "test"

    train_path.mkdir(parents=True, exist_ok=True)
    validation_path.mkdir(parents=True, exist_ok=True)
    test_path.mkdir(parents=True, exist_ok=True)

    pd.DataFrame(train).to_csv(train_path / "train.csv", header=False, index=False)
    pd.DataFrame(validation).to_csv(validation_path / "validation.csv", header=False, index=False)
    pd.DataFrame(test).to_csv(test_path / "test.csv", header=False, index=False)


def save_pipeline(base_dir, pipeline):
    """
    Saves the Scikit-Learn pipeline that we used to
    preprocess the data.
    """
    pipeline_path = Path(base_dir) / "pipeline"
    pipeline_path.mkdir(parents=True, exist_ok=True)
    dump(pipeline, open(pipeline_path / "pipeline.pkl", 'wb'))


def generate_baseline(base_dir, X_train, y_train):
    """
    Generates a baseline for our model using the train set.
    It saves the baseline in a JSON file where every line is
    a JSON object.
    """
    baseline_path = Path(base_dir) / "baseline"
    baseline_path.mkdir(parents=True, exist_ok=True)

    df = X_train.copy()
    df["label"] = y_train

    df.to_json(baseline_path / "baseline.json", orient='records', lines=True)


def preprocess(base_dir, data_filepath_train, data_filepath_test):
    """
    Preprocesses the supplied raw dataset and splits it into a train, validation,
    and a test set.
    """

    df_train = pd.read_csv(data_filepath_train, nrows=7200)
    df_test = pd.read_csv(data_filepath_test, nrows=2000)

    numerical_columns = df_train.select_dtypes(include=['number']).drop(['label'], axis=1).columns

    numerical_preprocessor = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", StandardScaler())
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ("numerical", numerical_preprocessor, numerical_columns),
        ]
    )

    X_train = df_train.copy()
    y_train = df_train['label']
    columns = list(X_train.drop(['label'], axis=1).columns)

    X_train, X_validation, y_train, y_validation =  train_test_split(X_train, y_train, test_size=0.2, random_state=12)
    X_test = df_test.copy()

    y_train = X_train.label
    y_validation = X_validation.label
    y_test = X_test.label

    X_train.drop(["label"], axis=1, inplace=True)
    X_validation.drop(["label"], axis=1, inplace=True)
    X_test.drop(["label"], axis=1, inplace=True)

    X_train = pd.DataFrame(X_train, columns=columns)
    X_validation = pd.DataFrame(X_validation, columns=columns)
    X_test = pd.DataFrame(X_test, columns=columns)

    y_train = y_train.astype(int)
    y_validation = y_validation.astype(int)
    y_test = y_test.astype(int)

    # Let's use the train set to generate a baseline that we can
    # later use to measure the quality of our model. This baseline
    # will use the original data.
    generate_baseline(base_dir, X_train, y_train)

    # Transform the data using the Scikit-Learn pipeline.
    X_train = preprocessor.fit_transform(X_train)
    X_validation = preprocessor.transform(X_validation)
    X_test = preprocessor.transform(X_test)

    train = np.concatenate((X_train, np.expand_dims(y_train, axis=1)), axis=1)
    validation = np.concatenate((X_validation, np.expand_dims(y_validation, axis=1)), axis=1)
    test = np.concatenate((X_test, np.expand_dims(y_test, axis=1)), axis=1)

    save_splits(base_dir, train, validation, test)
    save_pipeline(base_dir, pipeline=preprocessor)


if __name__ == "__main__":
    preprocess(BASE_DIR, DATA_FILEPATH_TRAIN, DATA_FILEPATH_TEST)

Overwriting /root/ml-school/mnist/preprocessor.py


In [ ]:
S3_FILEPATH = f"s3://mlschool-mnist/{MNIST_FOLDER}"


TRAIN_SET_S3_URI = sagemaker.s3.S3Uploader.upload(
    local_path=str(DATASET_FOLDER / "mnist_train.csv"), 
    desired_s3_uri=S3_FILEPATH,
)

TEST_SET_S3_URI = sagemaker.s3.S3Uploader.upload(
    local_path=str(DATASET_FOLDER / "mnist_test.csv"), 
    desired_s3_uri=S3_FILEPATH,
)

print(f"Train set S3 location: {TRAIN_SET_S3_URI}")
print(f"Test set S3 location: {TEST_SET_S3_URI}")

In [12]:
dataset_location_train = ParameterString(
    name="dataset_location_train",
    default_value=TRAIN_SET_S3_URI,
)

dataset_location_test = ParameterString(
    name="dataset_location_test",
    default_value=TEST_SET_S3_URI,
)

preprocessor_destination = ParameterString(
    name="preprocessor_destination",
    default_value=f"{S3_FILEPATH}/preprocessing",
)

baseline_destination = ParameterString(
    name="baseline_destination",
    default_value=f"{S3_FILEPATH}/baseline",
)



In [13]:
cache_config = CacheConfig(
    enable_caching=True, 
    expire_after="15d"
)

In [14]:
sklearn_processor = SKLearnProcessor(
    base_job_name="mnist-preprocessing",
    framework_version="0.23-1",
    instance_type="ml.t3.medium",
    instance_count=1,
    role=role
)

In [15]:
preprocess_step = ProcessingStep(
    name="preprocessing",
    processor=sklearn_processor,
    inputs=[
        ProcessingInput(source=dataset_location_train, destination="/opt/ml/processing/input/mnist_train"),
        ProcessingInput(source=dataset_location_test, destination="/opt/ml/processing/input/mnist_test"),
    ],
    outputs=[
        ProcessingOutput(output_name="train", source="/opt/ml/processing/train", destination=preprocessor_destination),
        ProcessingOutput(output_name="validation", source="/opt/ml/processing/validation", destination=preprocessor_destination),
        ProcessingOutput(output_name="test", source="/opt/ml/processing/test", destination=preprocessor_destination),
        ProcessingOutput(output_name="pipeline", source="/opt/ml/processing/pipeline", destination=preprocessor_destination),
        ProcessingOutput(output_name="baseline", source="/opt/ml/processing/baseline", destination=baseline_destination),
    ],
    code=f"{MNIST_FOLDER}/preprocessor.py",
    cache_config=cache_config
    
)

In [19]:
%%writefile {MNIST_FOLDER}/train.py

import os
import argparse

import numpy as np
import pandas as pd
import tensorflow as tf

from pathlib import Path
from sklearn.metrics import accuracy_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD


def train(base_directory, train_path, validation_path, epochs=50, batch_size=32, learning_rate=0.01):
    X_train = pd.read_csv(Path(train_path) / "train.csv")
    y_train = X_train[X_train.columns[-1]]
    X_train.drop(X_train.columns[-1], axis=1, inplace=True)

    X_validation = pd.read_csv(Path(validation_path) / "validation.csv")
    y_validation = X_validation[X_validation.columns[-1]]
    X_validation.drop(X_validation.columns[-1], axis=1, inplace=True)

    model = Sequential([
        Dense(10, input_shape=(X_train.shape[1],), activation="relu"),
        Dense(8, activation="relu"),
        Dense(10, activation="softmax"),
    ])

    model.compile(
        optimizer=SGD(learning_rate=learning_rate),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    model.fit(
        X_train, 
        y_train, 
        validation_data=(X_validation, y_validation),
        epochs=epochs, 
        batch_size=batch_size,
        verbose=2
    )

    predictions = np.argmax(model.predict(X_validation), axis=-1)
    print(f"Validation accuracy: {accuracy_score(y_validation, predictions)}")
    
    model_filepath = Path(base_directory) / "model" / "001"
    model.save(model_filepath)
    
if __name__ == "__main__":
    # Any hyperparameters provided by the training job are passed to the entry point
    # as script arguments. SageMaker will also provide a list of special parameters
    # that you can capture here. Here is the full list:
    # https://github.com/aws/sagemaker-training-toolkit/blob/master/src/sagemaker_training/params.py
    parser = argparse.ArgumentParser()
    parser.add_argument("--base_directory", type=str, default="/opt/ml/")
    parser.add_argument("--train_path", type=str, default=os.environ.get("SM_CHANNEL_TRAIN", None))
    parser.add_argument("--validation_path", type=str, default=os.environ.get("SM_CHANNEL_VALIDATION", None))
    parser.add_argument("--epochs", type=int, default=50)
    parser.add_argument("--batch_size", type=int, default=32)
    parser.add_argument("--learning_rate", type=float, default=0.1)
    args, _ = parser.parse_known_args()

    train(
        base_directory=args.base_directory,
        train_path=args.train_path,
        validation_path=args.validation_path,
        epochs=args.epochs,
        batch_size=args.batch_size,
        learning_rate=args.learning_rate
    )

Overwriting /root/ml-school/mnist/train.py


In [21]:
hyperparameters = {
    "epochs": 10,
    "batch_size": 32,
    "learning_rate": 0.1
}

use_spot_instances = True
max_run = 2400
max_wait = 3600 if use_spot_instances else None

estimator = TensorFlow(
    entry_point=f"{MNIST_FOLDER}/train.py",
    hyperparameters=hyperparameters,
    framework_version="2.8",
    py_version="py39",
    # instance_type="ml.m5.large",
    instance_type="ml.t3.medium",
    instance_count=1,
    script_mode=True,
    disable_profiler=True,
    role=role,
    use_spot_instances=use_spot_instances,
    max_run=max_run,
    max_wait=max_wait,
)

In [22]:
cache_config_train = CacheConfig(
    enable_caching=True, 
    expire_after="15d"
)

training_step = TrainingStep(
    name="training",
    estimator=estimator,
    inputs={
        "train": TrainingInput(
            s3_data=preprocess_step.properties.ProcessingOutputConfig.Outputs[
                "train"
            ].S3Output.S3Uri,
            content_type="text/csv"
        ),
        "validation": TrainingInput(
            s3_data=preprocess_step.properties.ProcessingOutputConfig.Outputs[
                "validation"
            ].S3Output.S3Uri,
            content_type="text/csv"
        )
    },
    cache_config=cache_config_train
)

In [23]:
hyperparameter_ranges = {
    "epochs": IntegerParameter(30, 35),
    "learning_rate": ContinuousParameter(0.1, 0.2)
}

objective_metric_name = "val_accuracy"
objective_type = "Maximize"
metric_definitions = [{"Name": objective_metric_name, "Regex": "Validation accuracy: ([0-9\\.]+)"}]
    
tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    objective_type=objective_type,
    max_jobs=4,
    max_parallel_jobs=1
)

In [24]:
cache_config_tunning = CacheConfig(
    enable_caching=True, 
    expire_after="15d"
)

tuning_step = TuningStep(
    name = "tuning",
    tuner=tuner,
    inputs={
        "train": TrainingInput(
            s3_data=preprocess_step.properties.ProcessingOutputConfig.Outputs[
                "train"
            ].S3Output.S3Uri,
            content_type="text/csv"
        ),
        "validation": TrainingInput(
            s3_data=preprocess_step.properties.ProcessingOutputConfig.Outputs[
                "validation"
            ].S3Output.S3Uri,
            content_type="text/csv"
        )
    },
    cache_config=cache_config_tunning
)

USE_TUNING_STEP = False

In [27]:
%%writefile {MNIST_FOLDER}/evaluation.py

import os
import json
import tarfile
import numpy as np
import pandas as pd

from pathlib import Path
from tensorflow import keras
from sklearn.metrics import accuracy_score
import argparse


MODEL_PATH = "/opt/ml/processing/model/"
TEST_PATH = "/opt/ml/processing/test/"


def evaluate(model_path, test_path, output_path):
    # The first step is to extract the model package provided
    # by SageMaker.
    with tarfile.open(Path(model_path) / "model.tar.gz") as tar:
        tar.extractall(path=Path(model_path))
        
    # We can now load the model from disk.
    model = keras.models.load_model(Path(model_path) / "001")
    
    X_test = pd.read_csv(Path(test_path) / "test.csv")
    y_test = X_test[X_test.columns[-1]]
    X_test.drop(X_test.columns[-1], axis=1, inplace=True)
    
    predictions = np.argmax(model.predict(X_test), axis=-1)
    accuracy = accuracy_score(y_test, predictions)
    print(f"Test accuracy: {accuracy}")

    # Let's add the accuracy of the model to our evaluation report.
    evaluation_report = {
        "metrics": {
            "accuracy": {
                "value": accuracy
            },
        },
    }
    
    # We need to save the evaluation report to the output path.
    Path(output_path).mkdir(parents=True, exist_ok=True)
    with open(Path(output_path) / "evaluation.json", "w") as f:
        f.write(json.dumps(evaluation_report))


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--output_path", type=str, default="/opt/ml/processing/evaluation/")
    args, _ = parser.parse_known_args()
    
    evaluate(
        model_path=MODEL_PATH,
        test_path=TEST_PATH,
        output_path=args.output_path
    )

Overwriting /root/ml-school/mnist/evaluation.py


In [30]:
# This is the input in case we want to use the best model generated
# by the Tuning Step.
tuning_model_input = ProcessingInput(
    source=tuning_step.get_top_model_s3_uri(
        top_k=1, 
        s3_bucket=sagemaker_session.default_bucket()
    ),
    destination="/opt/ml/processing/model",
)

# This is the input in case we want to use the trained model
# from the Training Step.
training_model_input = ProcessingInput(
    source=training_step.properties.ModelArtifacts.S3ModelArtifacts,
    destination="/opt/ml/processing/model"
)

# We can now select the appropriate input depending on which step
# we are using.
model_input = tuning_model_input if USE_TUNING_STEP else training_model_input

In [ ]:
tensorflow_processor = TensorFlowProcessor(
    framework_version="2.6",
    py_version="py38",
    base_job_name="mnist-evaluation-processor",
    instance_type="ml.t3.medium",
    instance_count=1,
    role=role,
)

# By default, the TensorFlowProcessor runs the script using
# /bin/bash as its entrypoint. We want to ensure we run it 
# using python3.
tensorflow_processor.framework_entrypoint_command = ["python3"]

In [ ]:
evaluation_destination = ParameterString(
    name="evaluation_destination",
    default_value=f'{S3_FILEPATH}/evaluation',
)

cache_config = CacheConfig(
    enable_caching=True, 
    expire_after="15d"
)

In [31]:
# We want to map the evaluation report that we generate inside
# the evaluation script so we can later reference it.
evaluation_report = PropertyFile(
    name="evaluation-report",
    output_name="evaluation",
    path="evaluation.json"
)


# Notice how this step uses the model generated by the tuning or training
# step, and the test set generated by the preprocessing step.
evaluation_step = ProcessingStep(
    name="evaluation_tensor",
    processor=tensorflow_processor,
    inputs=[
        model_input,
        ProcessingInput(
            source=preprocess_step.properties.ProcessingOutputConfig.Outputs[
                "test"
            ].S3Output.S3Uri,
            destination="/opt/ml/processing/test"
        )
    ],
    outputs=[
        ProcessingOutput(output_name="evaluation",
                         source="/opt/ml/processing/evaluation",
                         destination=evaluation_destination),
    ],
    code=f"{MNIST_FOLDER}/evaluation.py",
    property_files=[evaluation_report],
    cache_config=cache_config,
    job_arguments=["--output_path", "/opt/ml/processing/evaluation/"]
)



In [36]:
# This is the model data in case we want to use the best model generated
# by the Tuning Step.
tuning_model_data = tuning_step.get_top_model_s3_uri(
    top_k=0, 
    s3_bucket=sagemaker_session.default_bucket()
)

# This is the model data in case we want to use the trained model
# from the Training Step.
training_model_data = training_step.properties.ModelArtifacts.S3ModelArtifacts

# We can now select the appropriate model data depending on which step
# we are using.
model_data = tuning_model_data if USE_TUNING_STEP else training_model_data

In [37]:
model = TensorFlowModel(
    model_data=model_data,
    framework_version="2.8",
    sagemaker_session=PipelineSession(),
    role=role,
)

In [38]:
model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri=Join(on="", values=[
            evaluation_step.arguments['ProcessingOutputConfig']['Outputs'][0]['S3Output']['S3Uri'],
            "/evaluation.json"]
        ),
        content_type="application/json",
    )
)

In [39]:
model_package_group_name = "mnist-model-package-group"

register_model_step = ModelStep(
    name="register-model",
    step_args=model.register(
        content_types=["text/csv"],
        response_types=["text/csv"],
        inference_instances=["ml.t3.medium"],
        transform_instances=["ml.t3.medium"],
        domain="MACHINE_LEARNING",
        task="CLASSIFICATION",
        framework="TENSORFLOW",
        framework_version="2.6",
        # sample_payload_url="",
        model_package_group_name=model_package_group_name,
        model_metrics=model_metrics,
        approval_status=model_approval_status,
    ),
)

pa_model_step = ModelStep(
    name="pending-approval-step",
    step_args=model.register(
        content_types=["text/csv"],
        response_types=["text/csv"],
        inference_instances=["ml.t3.medium"],
        transform_instances=["ml.t3.medium"],
        domain="MACHINE_LEARNING",
        task="CLASSIFICATION",
        framework="TENSORFLOW",
        framework_version="2.6",
        # sample_payload_url="",
        model_package_group_name=model_package_group_name,
        model_metrics=model_metrics,
        approval_status=model_pending_approval_status,
    ),
)

/opt/conda/lib/python3.7/site-packages/sagemaker/workflow/pipeline_context.py:273: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  UserWarning,


In [40]:
from sagemaker.workflow.lambda_step import LambdaStep
from sagemaker.lambda_helper import Lambda

step_lambda = LambdaStep(
    name="ProcessingLambda",
    lambda_func=Lambda(
        function_arn="arn:aws:lambda:eu-north-1:284415450706:function:my-example-function"
    )
)

In [ ]:
model_approval_status = ParameterString(
    name="model_approval_status", 
    default_value="Approved"
)

accuracy_threshold = ParameterFloat(
    name="accuracy_threshold", 
    default_value=0.8
)

model_pending_approval_status = ParameterString(
    name="model_approval_status_pa", 
    default_value="PendingManualApproval"
)

accuracy_threshold_pa = ParameterFloat(
    name="accuracy_threshold", 
    default_value=0.50
)

In [131]:
# We can get the model accuracy directly from the evaluation
# report property file.
condition_gte = ConditionGreaterThanOrEqualTo(
    left=JsonGet(
        step_name=evaluation_step.name,
        property_file=evaluation_report,
        json_path="metrics.accuracy.value"
    ),
    right=accuracy_threshold
)

# If the condition succeeds, we can call the Model Step.
condition_step = ConditionStep(
    name="check-model-accuracy",
    conditions=[condition_gte],
    if_steps=[register_model_step],
    else_steps=[step_lambda],
)


condition_gte_pa = ConditionGreaterThanOrEqualTo(
    left=JsonGet(
        step_name=evaluation_step.name,
        property_file=evaluation_report,
        json_path="metrics.accuracy.value"
    ),
    right=accuracy_threshold_pa
)

# If the condition succeeds, we can call the Model Step.
condition_step_pa = ConditionStep(
    name="check-model-accuracy_pa",
    conditions=[condition_gte_pa],
    if_steps=[pa_model_step],
    else_steps=[]
)

fail_step = FailStep(
    name="fail",
    error_message=Join(
        on=" ", 
        values=[
            "Execution failed because the model's accuracy was lower than", 
            accuracy_threshold_pa
        ]
    ),
)

In [44]:
def get_latest_approved_model_package(model_package_group_name):
    """
    Returns the latest approved model package registered under the 
    specified model package group.
    """
    try:
        # We can use the boto3 SageMaker's API to list the existing
        # model packages with the specified name. We only care about
        # approved models.
        response = sagemaker_client.list_model_packages(
            ModelPackageGroupName=model_package_group_name,
            ModelApprovalStatus="Approved",
            SortBy="CreationTime",
            MaxResults=20,
        )
        approved_packages = response["ModelPackageSummaryList"]

        # If we get a NextToken back, we need to deal with pagination.
        while len(approved_packages) == 0 and "NextToken" in response:
            response = sagemaker_client.list_model_packages(
                ModelPackageGroupName=model_package_group_name,
                ModelApprovalStatus="Approved",
                SortBy="CreationTime",
                MaxResults=20,
                NextToken=response["NextToken"],
            )
            approved_packages.extend(response["ModelPackageSummaryList"])

        if len(approved_packages) == 0:
            print(f"No approved model pacakages for \"{model_package_group_name}\"")
            return None

        # At this point we identified the latest approved model,
        # so we can return it.
        print(f"Latest approved model package: {approved_packages[0]['ModelPackageArn']}")
        return approved_packages[0]

    except ClientError as e:
        print(e.response["Error"]["Message"])
        raise Exception(e.response["Error"]["Message"])

In [45]:
approved_model_package = get_latest_approved_model_package(model_package_group_name)
model_description = None

if approved_model_package:
    approved_model_package_arn = approved_model_package["ModelPackageArn"]

    model_description = sagemaker_client.describe_model_package(
        ModelPackageName=approved_model_package_arn
    )

model_description

No approved model pacakages for "mnist-model-package-group"


In [49]:
endpoint_name = "mnist-endpoint"

In [46]:
# model_package = ModelPackage(
#     model_package_arn=approved_model_package_arn, 
#     sagemaker_session=sagemaker_session,
#     role=role, 
# )

# model_package.deploy(
#     endpoint_name=endpoint_name,
#     initial_instance_count=1, 
#     instance_type="ml.m5.large", 
# )

NameError: name 'approved_model_package_arn' is not defined

In [50]:
import io
import csv

def preprocess_image(image_data):
    img_array = np.array(image_data)
    return img_array

def numpy_array_to_csv_bytes(array):
    csv_buffer = io.StringIO()
    writer = csv.writer(csv_buffer)
    writer.writerows(array)
    csv_buffer.seek(0)
    return csv_buffer.getvalue().encode()

In [51]:
import json
import numpy as np


# predictor = Predictor(endpoint_name=endpoint_name)

# # The payload we need to provide the model is in CSV format. Notice how the model expects data that's
# # already transformed. We can't provide the original data from our dataset because the model will not
# # work with it.

# df = pd.read_csv(DATASET_FOLDER / "mnist_test.csv", nrows=2)
# test = df.iloc[:, 1:] 
# img_array = preprocess_image(test)

# response = predictor.predict(numpy_array_to_csv_bytes(img_array), initial_args={"ContentType": "text/csv"})

# # We can decode the output of the endpoint and print the "predictions" key.
# predictions = json.loads(response.decode("utf-8"))["predictions"]
# print(f"Prediction: {np.argmax(predictions, axis=1)}")

ValidationError: An error occurred (ValidationError) when calling the InvokeEndpoint operation: Endpoint mnist-endpoint of account 284415450706 not found.

In [53]:
from sagemaker.tensorflow.model import TensorFlowModel
from sagemaker.tensorflow.model import TensorFlowPredictor
from sagemaker.workflow.lambda_step import LambdaStep, LambdaOutput, LambdaOutputTypeEnum
from sagemaker.workflow.parameters import ParameterBoolean
from sagemaker.lambda_helper import Lambda
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
from sagemaker.s3 import S3Downloader

In [84]:
CODE_FOLDER = f"{MNIST_FOLDER}/code"

import os

CODE_FOLDER = "/root/ml-school/mnist/codepipe"

# Create the directory if it does not exist
if not os.path.exists(CODE_FOLDER):
    os.makedirs(CODE_FOLDER)

In [104]:
%%writefile {CODE_FOLDER}/inference.py

import os
import json
import boto3
import requests
import numpy as np
import pandas as pd
import io

from pickle import load
from pathlib import Path


PIPELINE_FILE = Path("/tmp") / "pipeline.pkl"

# By default, we will read the S3 location of the Scikit-Learn
# pipeline from an environment variable that we'll configure
# when registering the model.
PIPELINE_S3_LOCATION = os.environ.get("PIPELINE_S3_LOCATION", None)


s3 = boto3.resource("s3")


def handler(data, context, pipeline_s3_location=PIPELINE_S3_LOCATION):
    """
    This is the entrypoint that will be called by SageMaker when the endpoint
    receives a request. You can see more information at 
    https://github.com/aws/sagemaker-tensorflow-serving-container.
    """
    print("Handling endpoint request")
    
    download_pipeline(pipeline_s3_location)
    
    instance = _process_input(data, context)
    output = _predict(instance, context)
    return _process_output(output, context)


def download_pipeline(pipeline_s3_location):
    """
    This function will download the Scikit-Learn pipeline from S3
    if it doesn't already exist. We need the pipeline to pre-process
    the data before we run it through the model.
    """
    
    print(f"pipiline s3 lcoation: {pipeline_s3_location}")
    
    s3_parts = pipeline_s3_location.split('/', 3)
    bucket = s3_parts[2]
    key = s3_parts[3]
    
    if not PIPELINE_FILE.exists():
        s3.Bucket(bucket).download_file(f"{key}/pipeline.pkl", str(PIPELINE_FILE))


def transform(payload):
    """
    This function transforms the payload in the request using the
    Scikit-Learn pipeline that we created during the preprocessing step.
    """
    
    print("Transforming input data...")
    # print(f"Payload: {payload} \n")
    
    pipeline = load(open(PIPELINE_FILE, 'rb'))
    
    image = payload.get("image", "")
    
#     data = pd.DataFrame(
#         # columns=["island", "culmen_length_mm", "culmen_depth_mm", "flipper_length_mm", "body_mass_g"], 
#             image
        
#     )
    
    print(f"data: {image.shape} \n")
    
    result = pipeline.transform(image)
    
    print(f"result: {result} \n")
    return result[0].tolist()
    

def _process_input(data, context):
    print("Processing input data...")
    
    if context is None:
        # The context will be None when we are testing the code
        # directly from a notebook. In that case, we can use the
        # data directly.
        endpoint_input = data
    elif context.request_content_type in ("application/json", "application/octet-stream"):
        # When the endpoint is running, we will receive a context
        # object. We need to parse the input and turn it into 
        # JSON in that case.
        print("context is JSON.")
        endpoint_input = json.loads(data.read().decode("utf-8"))

        if endpoint_input is None:
            raise ValueError("There was an error parsing the input request.")
            
    elif context.request_content_type is "application/csv":
        # When the endpoint is running, we will receive a context
        # object. We need to parse the input and turn it into 
        # CSV in that case.
        print("context is CSV.")
        endpoint_input = data_bytesio = io.BytesIO(data_csv.encode("utf-8"))

        if endpoint_input is None:
            raise ValueError("There was an error parsing the input request.")
    else:
        print("unsupported type data")
        raise ValueError(f"Unsupported content type: {context.request_content_type or 'unknown'}")
        
    return transform(endpoint_input)


def _predict(instance, context):
    print("Sending input data to model to make a prediction...")
    
    model_input = json.dumps({"instances": [instance]})
    
    print(f"model input: {model_input}")
    
    if context is None:
        # The context will be None when we are testing the code
        # directly from a notebook. In that case, we want to return
        # a fake prediction back.
        result = {
            "predictions": [
                [0.2, 0.5, 0.3]
            ]
        }
    else:
        # When the endpoint is running, we will receive a context
        # object. In that case we need to send the instance to the
        # model to get a prediction back.
        response = requests.post(context.rest_uri, data=model_input)
        
        if response.status_code != 200:
            raise ValueError(response.content.decode('utf-8'))
            
        result = json.loads(response.content)
    
    print(f"Response: {result}")
    return result


def _process_output(output, context):
    print("Processing prediction received from the model...")
    
    response_content_type = "application/json" if context is None else context.accept_header
    
    prediction = np.argmax(output["predictions"][0])
    confidence = output["predictions"][0][prediction]
    
    print(f"Prediction: {prediction}. Confidence: {confidence}")
    
    result = json.dumps({
        "prediction": str(prediction),
        "confidence": confidence
    }), response_content_type
    
    return result

Overwriting /root/ml-school/mnist/codepipe/inference.py


In [111]:
from codepipe.inference import handler

df = pd.read_csv(DATASET_FOLDER / "mnist_test.csv", nrows=2)
test = df.iloc[:1, 1:] 
img_array = preprocess_image(test)


# handler(
#     data={
#         "image": test,
#     }, 
#     context=None, 
#     pipeline_s3_location=preprocessor_destination.default_value
# )

In [112]:
model = TensorFlowModel(
    model_data=model_data,
    entry_point="inference.py",
    source_dir=str(CODE_FOLDER),
    env={
        "PIPELINE_S3_LOCATION": preprocessor_destination,
    },
    framework_version="2.6",
    sagemaker_session=PipelineSession(),
    role=role,
)


register_model_step = ModelStep(
    name="register-model",
    step_args=model.register(
        content_types=["application/json"],
        response_types=["application/json"],
        inference_instances=["ml.m5.large"],
        domain="MACHINE_LEARNING",
        task="CLASSIFICATION",
        framework="TENSORFLOW",
        framework_version="2.6",
        model_package_group_name=model_package_group_name,
        model_metrics=model_metrics,
        approval_status=model_approval_status,
    )
)

/opt/conda/lib/python3.7/site-packages/sagemaker/workflow/pipeline_context.py:273: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  UserWarning,


In [113]:
data_capture_enabled = ParameterBoolean(
    name="data_capture_enabled",
    default_value=True,
)

data_capture_percentage = ParameterInteger(
    name="data_capture_percentage",
    default_value=100,
)

data_capture_destination = ParameterString(
    name="data_capture_destination",
    default_value=f"{S3_FILEPATH}/monitoring/data-capture",
)

In [114]:
%%writefile {MNIST_FOLDER}/lambda.py

import os
import json
import boto3

sagemaker = boto3.client("sagemaker")

def lambda_handler(event, context):
    model_name = event["model_name"]
    endpoint_name = event["endpoint_name"]
    endpoint_config_name = event["endpoint_config_name"]
    data_capture_enabled = event["data_capture_enabled"]
    data_capture_percentage = event["data_capture_percentage"]
    data_capture_destination = event["data_capture_destination"]
    role = event["role"]
    
    
    # The first step is to create a new model. We will use the
    # ARN of the model we registered.
    sagemaker.create_model(
        ModelName=model_name, 
        ExecutionRoleArn=role, 
        Containers=[{
            "ModelPackageName": event["model_package_arn"]
        }] 
    )

    # Then, we need to create an Endpoint Configuration.
    # Here is where we specify the hardware we need for the
    # endpoint and the Data Capture configuration. 
    sagemaker.create_endpoint_config(
        EndpointConfigName=endpoint_config_name,
        ProductionVariants=[
            {
                "ModelName": model_name,
                "InstanceType": "ml.t3.medium",
                # "InstanceType": "ml.m5.large",
                "InitialVariantWeight": 1,
                "InitialInstanceCount": 1,
                "VariantName": "AllTraffic",
            }
        ],
        DataCaptureConfig={
            "EnableCapture": data_capture_enabled,
            "InitialSamplingPercentage": data_capture_percentage,
            "DestinationS3Uri": data_capture_destination,
            "CaptureOptions": [
                {
                    'CaptureMode': "Input"
                },
                {
                    'CaptureMode': "Output"
                },
            ],
            "CaptureContentTypeHeader": {
                "JsonContentTypes": [
                    "application/json",
                    "application/octect-stream",
                    "application/csv"
                ]
            }
        },
    )

    # Finally, we can create the new Endpoint using the
    # Endpoint Configuration we created above.
    sagemaker.create_endpoint(
        EndpointName=endpoint_name, 
        EndpointConfigName=endpoint_config_name,
    )
    
    return {
        "statusCode": 200,
        "body": json.dumps("Endpoint deployed successfully"),
        "model_name": model_name,
    }

Writing /root/ml-school/mnist/lambda.py


In [115]:
# Here should be role but I 

In [126]:
import time

timestamp_signature = ParameterString(
    name="timestamp_signature",
    default_value="",
)

function_name = f"deploy-endpoint-fn-{time.strftime('%m%d%H%M%S', time.localtime())}"
endpoint_name = "mnist-endpoint"

deploy_step = LambdaStep(
    name="deploy",
    lambda_func=Lambda(
        function_name=function_name,
        # execution_role_arn=lambda_role,
        execution_role_arn=role,
        script=str(Path(MNIST_FOLDER) / "lambda.py"),
        handler="lambda.lambda_handler",
        timeout=600,
        memory_size=10240,
    ),
    inputs={
        # We can use the timestamp_signature pipeline parameter
        # to add a suffix to the model and the endpoint configuration
        # and avoid name collisions.
        "model_name": Join(on="-", values=["mnist-model", timestamp_signature]),
        "endpoint_config_name": Join(on="-", values=["mnist-endpoint-config", timestamp_signature]),

        # We use the ARN of the model we registered to
        # deploy it to the endpoint.
        "model_package_arn": register_model_step.properties.ModelPackageArn,

        "endpoint_name": endpoint_name,
        "data_capture_enabled": data_capture_enabled,
        "data_capture_percentage": data_capture_percentage,
        "data_capture_destination": data_capture_destination,
        "role": role,
    },
    outputs=[
        LambdaOutput(output_name="model_name", output_type=LambdaOutputTypeEnum.String)
    ]
)

In [132]:
condition_step = ConditionStep(
    name="check-model-accuracy",
    conditions=[condition_gte],
    if_steps=[
        register_model_step, deploy_step
    ],
    else_steps=[fail_step], 
)

In [ ]:
session5_pipeline = Pipeline(
    name="penguins-session5-pipeline",
    parameters=[
        dataset_location_train,
        dataset_location_test, 
        preprocessor_destination,
        baseline_destination,
        evaluation_destination,
        model_approval_status,
        model_pending_approval_status
        accuracy_threshold,
        accuracy_threshold_pa
        data_capture_enabled,
        data_capture_percentage,
        data_capture_destination,
    ],
    steps=[
        preprocess_step, 
        tuning_step if USE_TUNING_STEP else training_step, 
        evaluation_step,
        condition_step,
        condition_step_pa
    ],
)

In [ ]:
session5_pipeline.upsert(role_arn=role)

execution = session5_pipeline.start(parameters=dict(
    timestamp_signature=time.strftime("%m%d%H%M%S", time.localtime())
))

In [ ]:
for mp in sagemaker_client.list_model_packages(ModelPackageGroupName=model_package_group_name)["ModelPackageSummaryList"]:
    print(f"Deleting {mp['ModelPackageArn']}")
    sagemaker_client.delete_model_package(ModelPackageName=mp["ModelPackageArn"])

# We can now delete the model package group.    
sagemaker_client.delete_model_package_group(ModelPackageGroupName=model_package_group_name)

# And finally we delete the endpoint and the pipeline.
predictor.delete_endpoint()
session4_pipeline.delete()